In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("buffer/resnet50_resnext50_32x4d_vgg16_wide_resnet50_2/trace_1024.json")
print(df)
print(df.columns)

                      layer   src           dst  interval  flit  counts  \
0           resnet50_layer1  [-1]  [0, 1, 2, 3]     10976    37      56   
9           resnet50_layer2  [-1]        [0, 2]       448    30      28   
10          resnet50_layer2  [-1]        [1, 3]       448    30      28   
19          resnet50_layer3  [-1]        [0, 1]      4032    92      56   
48          resnet50_layer6  [-3]  [0, 1, 2, 3]       512    67      98   
..                      ...   ...           ...       ...   ...     ...   
182  wide_resnet50_2_layer4  [-3]           [3]      3136    11      32   
183  wide_resnet50_2_layer4  [-1]           [0]     25088   198       4   
184  wide_resnet50_2_layer4  [-1]           [1]     25088   198       4   
185  wide_resnet50_2_layer4  [-1]           [2]     25088   198       4   
186  wide_resnet50_2_layer4  [-1]           [3]     25088   198       4   

    datatype  delay map_src           map_dst  captain  \
0      input    NaN     [8]  [66, 67, 82,

In [19]:
communicate_time = (df["issue_time"] + df["flit"]).quantile(0.9)
print(communicate_time)
compute_time = (df["interval"] * df["counts"]).quantile(0.9)
print(compute_time)
slowdown = communicate_time / compute_time
print(slowdown)
fk = df.iloc[0].apply(lambda x: (x["layer"], x["src"] + x["dst"]), axis=1)
print(fk)
for i, j in fk:
    print(i, j)

615506.3333333334
614656.0
1.0013834296473691


TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [32]:
df = pd.read_csv("focus-final-out/baseline_bert-large_inception_unet_wide_resnet50_2.csv", index_col=0)
a = pd.Series(dict(zip(df.columns, ["fk", 512, 100, 1])))
print(a)
df[df["flit"] == 1024] = a
print(df)

name         fk
flit        512
cycle       100
slowdown      1
dtype: object


ValueError: Must have equal len keys and value when setting with an iterable

In [68]:
df = pd.read_json("buffer/resnet50_resnext50_32x4d_vgg16_wide_resnet50_2/solution_1024.json")
df2 = pd.read_json("buffer/resnet50_resnext50_32x4d_vgg16_wide_resnet50_2/solution_2048.json")
c1 = (df["issue_time"] + df["flit"]).quantile(0.9)
m1 = (df["counts"] * df["interval"]).quantile(0.9)
print(m1)

c2 = (df2["issue_time"] + df2["flit"]).quantile(0.9)
print(c1, c2)

3145728.0
3145758.0 3145733.0


In [34]:
pd.read_json("buffer/resnet50_resnext50_32x4d_vgg16_wide_resnet50_2/trace_2048.json")

,layer,src,dst,interval,flit,counts,datatype,delay,map_src,map_dst,captain,tree,epfl
9,resnet50_layer2,[-1],"[0, 2]",448,16,28,input,NaN,[123],"[74, 90]",74.0,"[[74, 3], [74, 1], [90, 1]]",1.0
19,resnet50_layer3,[-1],"[0, 1]",4032,47,56,input,NaN,[91],"[72, 73]",72.0,"[[72, 5], [72, 1], [73, 1]]",1.0
57,resnet50_layer7,[-1],"[0, 1]",4032,47,56,input,NaN,[185],"[170, 171]",170.0,"[[170, 5], [170, 1], [171, 1]]",1.0
83,resnet50_layer10,[-1],"[0, 1]",4032,47,56,input,NaN,[157],"[142, 143]",142.0,"[[142, 5], [142, 1], [143, 1]]",1.0
109,resnet50_layer13,[-3],"[0, 2]",7056,21,32,weight,NaN,[247],"[204, 220]",204.0,"[[204, 3], [204, 1], [220, 1]]",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,wide_resnet50_2_layer16,[3],[-2],256,2,392,output,256.0,[125],[128],NaN,[],NaN
520,wide_resnet50_2_layer16,[-3],[0],100352,131,1,weight,NaN,[127],[108],NaN,[],NaN
521,wide_resnet50_2_layer16,[-3],[1],100352,131,1,weight,NaN,[127],[109],NaN,[],NaN
522,wide_resnet50_2_layer16,[-3],[2],100352,131,1,weight,NaN,[127],[124],NaN,[],NaN


In [86]:
df = pd.read_json("buffer/resnet50_resnext50_32x4d_vgg16_wide_resnet50_2/solution_512.json")
df2 = pd.read_json("buffer/resnet50_resnext50_32x4d_vgg16_wide_resnet50_2/solution_2048.json")

# print(df[df["delay"] > 0])

res = df.groupby(by="layer").apply(lambda x: ((x["delay"] + x["interval"]) * x["counts"]).max())
res = res.quantile(0.9)

res2 = df2.groupby(by="layer").apply(lambda x: ((x["delay"] + x["interval"]) * x["counts"]).max())
res2 = res2.quantile(0.9)
print(res, res2)

1201034.2400000168 613401.6


In [7]:
import pandas as pd
df = pd.read_json("buffer/bert/trace_1024.json")
print(df)

          layer   src   dst  interval  flit  counts datatype  delay map_src  \
0   bert_layer1   [0]  [-2]        64     3      18   output   64.0     [0]   
1   bert_layer1   [1]  [-2]        64     3      18   output   64.0    [16]   
2   bert_layer1  [-3]   [0]        64   131      18   weight    NaN   [112]   
3   bert_layer1  [-3]   [1]        64   131      18   weight    NaN   [112]   
4   bert_layer1  [-1]   [0]        64     3      18    input    NaN     [8]   
5   bert_layer1  [-1]   [1]        64     3      18    input    NaN     [8]   
6   bert_layer2   [0]  [-2]        64     3      18   output   64.0     [1]   
7   bert_layer2   [1]  [-2]        64     3      18   output   64.0    [17]   
8   bert_layer2  [-3]   [0]        64   131      18   weight    NaN   [128]   
9   bert_layer2  [-3]   [1]        64   131      18   weight    NaN   [128]   
10  bert_layer2  [-1]   [0]        64     3      18    input    NaN    [16]   
11  bert_layer2  [-1]   [1]        64     3      18 

In [6]:
import re
def prelayer(layer):
    layer_number = re.findall(r"\d+", layer)[-1]
    pre_layer_number = str(int(layer_number) - 1)
    pre_layer = re.sub(layer_number[::-1], pre_layer_number[::-1], layer[::-1])[::-1]
    return pre_layer

layer = "bert128_7654"
print(prelayer(layer))


bert128_7653
